In [1]:
from models.model_LIT_CODING import LITIlluminationModel
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from dataset.dataset_utils import SimulatedLabelModule
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import yaml

In [2]:

photon_count = 200
sbr = 1.0

n_tbins = 200
k = 4
sigma = 10

yaml_file = 'config/best_hyperparameters_illumination_v1.yaml'
log_dir = 'experiments'

In [3]:

logger = TensorBoardLogger(log_dir, name="illum_models")

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{log_dir}/{logger.name}/version_{logger.version}/checkpoints",  
    filename = 'coded_model'# Directory to save the model
    save_top_k=1,  # Save only the best model
    monitor="val_loss",  # Metric to monitor
    mode="min",  # Minimize the monitored metric
)


In [4]:
try:
    with open(yaml_file, 'r') as file:
        config = yaml.safe_load(file)

    init_lr = config['init_lr']
    lr_decay_gamma = config['lr_decay_gamma']
    tv_reg = config['tv_reg']
    epochs = config['epochs']
    batch_size = config['batch_size']
    beta = config['beta']
    num_samples = config['num_samples']
except (FileNotFoundError, TypeError) as e:
    print(e)
    exit(0)


In [5]:
print(tv_reg)

0.0011869485344305597


In [ ]:
label_module = SimulatedLabelModule(n_tbins, batch_size=batch_size, num_samples=num_samples)
label_module.setup()

print(len(label_module.train_dataset))


24141


In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.set_float32_matmul_precision('medium')

else:
    device = torch.device("cpu")

pl.seed_everything(42)


Seed set to 42


42

In [8]:
trainer = pl.Trainer(logger=logger, max_epochs=epochs,
                      log_every_n_steps=250, val_check_interval=0.5,
                      callbacks=[checkpoint_callback])


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
lit_model = LITIlluminationModel(k=k, n_tbins=n_tbins, init_lr=init_lr, lr_decay_gamma=lr_decay_gamma,
                           beta=beta, tv_reg=tv_reg, photon_count=photon_count, sbr=sbr, sigma=sigma)

torch.autograd.set_detect_anomaly(True)

In [ ]:
trainer.fit(lit_model, datamodule=label_module)

2025-01-20 13:59:50.277894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-20 13:59:50.317625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-20 13:59:50.329865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 13:59:50.358892: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  |

Epoch 46:   5%|▍         | 92/1857 [00:02<00:50, 34.76it/s, v_num=2]       